### This notebook reproduce an event: Experiment is infeasible: 'event: negative particle crack length larger than particle radius'. It seems until now that this cannot be resolved by increasing mesh points in R direction, but can be resolved by reducing cracking rate

In [58]:
import pybamm
import matplotlib.pyplot as plt
import numpy as np;  
import pandas as pd
import timeit
start = timeit.default_timer()

In [59]:
model = pybamm.lithium_ion.DFN(
    options={
    "SEI": "interstitial-diffusion limited", 
    "SEI on cracks": "true",
    "particle mechanics": "swelling and cracking",
    "SEI film resistance": "distributed", 
    "SEI porosity change": "true",
})

In [60]:
param = pybamm.ParameterValues("OKane2022")
param.update({'Negative electrode cracking rate': 5e-15})
var_pts = {
    "x_n": 5,  # negative electrode
    "x_s": 5,  # separator 
    "x_p": 5,  # positive electrode
    "r_n": 60,  # negative particle
    "r_p": 20,  # positive particle
}

In [61]:
total_cycles=8
param.update({'Negative electrode cracking rate': 2.8e-16})
exp = pybamm.Experiment( 
    [(
    "Discharge at 1C until 2.5V", 
    "Charge at 0.3C until 4.2V", 
    "Hold at 4.2V until C/100",
    ),] * total_cycles 
)

In [62]:
sim1 = pybamm.Simulation(
    model, 
    experiment=exp,
    parameter_values=param,
    solver=pybamm.CasadiSolver(
        return_solution_if_failed_early=True,),
    var_pts=var_pts,
)
solution1 = sim1.solve(calc_esoh=False)
stop = timeit.default_timer()
print('running time: ' + str(stop - start) +'s')

At t = 473.369 and h = 3.60802e-14, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 233.369 and h = 4.87395e-14, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 113.369 and h = 2.3805e-18, the corrector convergence failed repeatedly or with |h| = hmin.
2023-08-28 11:43:56.471 - [WARNING] callbacks.on_experiment_infeasible(231): 

	Experiment is infeasible: 'event: negative particle crack length larger than particle radius' was triggered during 'Discharge at 1C until 2.5V'. The returned solution only contains up to step 1 of cycle 8. 


running time: 15.165938499965705s


If we reduce the cracking rate, the problem will be resolved

In [63]:
param.update({'Negative electrode cracking rate': 2.8e-17})
sim1 = pybamm.Simulation(
    model, 
    experiment=exp,
    parameter_values=param,
    solver=pybamm.CasadiSolver(
        return_solution_if_failed_early=True,),
    var_pts=var_pts,
)
solution1 = sim1.solve(calc_esoh=False)

At t = 473.37, , mxstep steps taken before reaching tout.
At t = 233.37, , mxstep steps taken before reaching tout.
At t = 113.37 and h = 3.02027e-14, the corrector convergence failed repeatedly or with |h| = hmin.


Alternatively, we can increase negative electrode diffusivity. This is because cracking is determiend by both cracking rate and concentration gradient. Increasing diffusivity can reduce concentration gradient. 

In [64]:
param.update({'Negative electrode cracking rate': 2.8e-16})
param.update({"Negative electrode diffusivity [m2.s-1]": 3.3e-13})
sim1 = pybamm.Simulation(
    model, 
    experiment=exp,
    parameter_values=param,
    solver=pybamm.CasadiSolver(
        return_solution_if_failed_early=True,),
    var_pts=var_pts,
)
solution1 = sim1.solve(calc_esoh=False)

At t = 535.22 and h = 2.12029e-14, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 144.264 and h = 2.61363e-14, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 144.264 and h = 5.10505e-14, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 144.185 and h = 2.50788e-14, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 144.185, , mxstep steps taken before reaching tout.
At t = 144.124 and h = 2.99657e-14, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 144.124 and h = 6.7446e-15, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 144.073 and h = 1.49902e-14, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 144.073 and h = 1.4192e-14, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 144.027 and h = 2.65414e-14, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 144.027, repeated recoverable residua

However, this event cannot be resolved by increasing mesh points in R direction.

In [65]:
param.update({'Negative electrode cracking rate': 2.8e-16})
param.update({"Negative electrode diffusivity [m2.s-1]": 3.3e-14})
var_pts = {
    "x_n": 5,  # negative electrode
    "x_s": 5,  # separator 
    "x_p": 5,  # positive electrode
    "r_n": 100,  # negative particle
    "r_p": 20,  # positive particle
}
sim1 = pybamm.Simulation(
    model, 
    experiment=exp,
    parameter_values=param,
    solver=pybamm.CasadiSolver(
        return_solution_if_failed_early=True,),
    var_pts=var_pts,
)
solution1 = sim1.solve(calc_esoh=False)

At t = 473.357, , mxstep steps taken before reaching tout.
At t = 233.357 and h = 2.16731e-14, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 113.357 and h = 5.60958e-14, the corrector convergence failed repeatedly or with |h| = hmin.
2023-08-28 11:45:46.779 - [WARNING] callbacks.on_experiment_infeasible(231): 

	Experiment is infeasible: 'event: negative particle crack length larger than particle radius' was triggered during 'Discharge at 1C until 2.5V'. The returned solution only contains up to step 1 of cycle 8. 


one may think 100 is still not enough. However, if we give r_n=200, then it will give a solver error

In [66]:
var_pts = {
    "x_n": 5,  # negative electrode
    "x_s": 5,  # separator 
    "x_p": 5,  # positive electrode
    "r_n": 200,  # negative particle
    "r_p": 20,  # positive particle
}
sim1 = pybamm.Simulation(
    model, 
    experiment=exp,
    parameter_values=param,
    solver=pybamm.CasadiSolver(
        return_solution_if_failed_early=True,),
    var_pts=var_pts,
)
solution1 = sim1.solve(calc_esoh=False)

The residual function failed at the first call. 
The residual function failed at the first call. 
The residual function failed at the first call. 
The residual function failed at the first call. 
The residual function failed at the first call. 
The residual function failed at the first call. 
2023-08-28 11:46:33.169 - [ERROR] callbacks.on_experiment_error(224): Simulation error: Maximum number of decreased steps occurred at t=0.0 (final SolverError: 'Error in Function::call for 'F' [IdasInterface] at .../casadi/core/function.cpp:1401:
Error in Function::call for 'F' [IdasInterface] at .../casadi/core/function.cpp:330:
.../casadi/interfaces/sundials/idas_interface.cpp:596: IDACalcIC returned "IDA_FIRST_RES_FAIL". Consult IDAS documentation.'). For a full solution try reducing dt_max (currently, dt_max=600) and/or reducing the size of the time steps or period of the experiment. Set `return_solution_if_failed_early=True` to return the solution object up to the point where failure occured.

SolverError: Maximum number of decreased steps occurred at t=0.0 (final SolverError: 'Error in Function::call for 'F' [IdasInterface] at .../casadi/core/function.cpp:1401:
Error in Function::call for 'F' [IdasInterface] at .../casadi/core/function.cpp:330:
.../casadi/interfaces/sundials/idas_interface.cpp:596: IDACalcIC returned "IDA_FIRST_RES_FAIL". Consult IDAS documentation.'). For a full solution try reducing dt_max (currently, dt_max=600) and/or reducing the size of the time steps or period of the experiment. Set `return_solution_if_failed_early=True` to return the solution object up to the point where failure occured.